## Classification via Generative Models
(50%) We are going to explore the problem of identifying smartphone position through probabilistic generative models. Motion sensors in smartphones provide valuable information for researchers to understand its owners. An interesting (and more challenging) task is to identify human activities through the data recorded by motion sensors. For example, we want to know whether the smartphone owner is walking, running, or biking. In this homework problem, we are going to tackle a simpler problem. We want to know the static position of the smartphone. There are six possible positions:

Phoneonback: The phone is laying on the back of the phone with the screen pointing up (away from the ground).
Phoneonfront: The phone is laying on the back of the phone with the screen pointing towards the ground
Phoneonbottom: The phone is standing on the bottom of the screen, meaning the bottom is pointed towards the ground
Phoneontop: The phone is standing on the top of the screen, meaning the top is pointed towards the ground
Phoneonleft: The phone is laying on the left side of the screen.
Phoneonright: The phone is laying on the right side of the screen.
The input data is the reading of the accelerometer (cf. https://en.wikipedia.org/wiki/Accelerometer) in the smartphone. We have a training dataset that contains about 28,500 data points for phones in each of the six positions. The following is some basic information of the training dataset.

In [1]:
import pickle

with open('phone_train.pickle', 'rb') as fh1:
    traindata = pickle.load(fh1)
    
print("The first few rows are:")
print(traindata.head())
print("\nSummary statistics:")
print(traindata.describe())
print("\nLabel counts:")
print(traindata['label'].value_counts())

The first few rows are:
          x         y         z        label
0  0.138809  0.074341  9.801056  Phoneonback
1  0.164993  0.006500  9.690369  Phoneonback
2  0.211411 -0.001831  9.755829  Phoneonback
3  0.184036 -0.007782  9.774872  Phoneonback
4  0.142380  0.005310  9.765350  Phoneonback

Summary statistics:
                   x              y              z
count  167097.000000  167097.000000  167097.000000
mean        0.357340       0.146807      -0.015550
std         5.622396       5.552010       5.737150
min        -9.770279      -9.966507      -9.908417
25%         0.016617      -0.074875      -0.221497
50%         0.142776       0.009628       0.025223
75%         0.249893       0.295715       0.151032
max        10.073685       9.980255      10.031113

Label counts:
Phoneonleft      29522
Phoneonfront     29079
Phoneonback      28566
Phoneonbottom    27842
Phoneontop       26401
Phoneonright     25687
Name: label, dtype: int64


It is obvious that the last column, label, is the target of our predictive model and the first three columns are the input features.

We are going to train a model that can predict smartphone positions given its accelerometer readings. To achieve this goal, we are going to divide this question into two sub-questions. The first is about training a generative classifier, and the second is to predict smartphone positions.

Creat a Python class named mygpc for this task.

**Q1.1: Create your mypgc class.**

In [2]:
import math
import numpy as np
from collections import defaultdict
from prettytable import PrettyTable


class mygpc():
    
    def __init__(self):
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.amodel = defaultdict()
        self.classes = []
    
    def fit(self, x, y):
        self.X_train = np.array(x)
        self.Y_train = np.array(y)
        self.train()
    
    def train(self):
        
        if self.X_train is None or self.Y_train is None:
            raise ValueError("Please fit the training data first.")
            
        self.classes = np.unique(self.Y_train)
        for c in self.classes:
            indexes = np.where(self.Y_train == c)
            x_in_class = self.X_train[indexes]

            self.amodel[c] = dict()
            self.amodel[c]["mu"] = np.mean(x_in_class, axis=0)
            self.amodel[c]["cov"] = np.cov(x_in_class, rowvar=False)
            
            try:
                self.amodel[c]["prec"] = np.linalg.inv(self.amodel[c]["cov"])
            except numpy.linalg.LinAlgError:
                raise ValueError("Covariance Matrix for class {} is not invertible.".format(c))
            
            self.amodel[c]["detcov"] = math.log(np.linalg.det(self.amodel[c]["cov"]))
            self.amodel[c]["n"] = len(x_in_class)
            self.amodel[c]["prior"] = len(x_in_class)/len(self.X_train)
        
        # find Wk and Wk0 for each class k
        self.shared_cov = self.ML_cov()
        try:
            self.shared_cov_inv = np.linalg.inv(self.shared_cov)
        except numpy.linalg.LinAlgError:
            raise ValueError("Shared Covariance Matrix is not invertible.")
        
        for c in self.classes:
            self.amodel[c]["W"] = np.dot(self.shared_cov_inv, self.amodel[c]["mu"].transpose())
            self.amodel[c]["W0"] = np.dot(np.dot(self.amodel[c]["mu"], self.shared_cov_inv),
                                          self.amodel[c]["mu"].transpose())*(-0.5)+math.log(self.amodel[c]["prior"])
        
        
    
    def show_model_parameters(self):
        
        np.set_printoptions(precision=3, suppress=True)
        
        par_names = ["mu", "cov", "prec", "detcov", "n", "prior"]
        for c in self.classes:
            t = PrettyTable(par_names[:-3]+["Others"])
            par_value = []
            for par in par_names[:-3]:
                par_value.append(self.amodel[c][par])
            par_value.append("detcov: {}\nn: {}\nprior: {}".format(self.amodel[c]["detcov"],
                                                                   self.amodel[c]["n"],
                                                                   self.amodel[c]["prior"]))

            t.add_row(par_value)
            t.align = 'l'
            print(c)
            print(t)
            print()
    
    def ML_cov(self):
        """Return Shared Covariance Matrix by Maximum Likelihood."""
        shared_cov = np.zeros(self.amodel[self.classes[0]]["cov"].shape)
        for c in self.classes:
            shared_cov += (self.amodel[c]["prior"] * self.amodel[c]["cov"])
        
        return(shared_cov)
            
    
    def predict_one(self, x_test):
        exp_aks = dict()
        posteriors = dict()
        sum_exp_aks = 0
        
        best_ak = 0
        best_class = str()
        for c in self.classes:
            ak = np.dot(self.amodel[c]["W"], x_test) + self.amodel[c]["W0"]
            if ak > best_ak:
                best_ak = ak
                best_class = c

#         for c in self.classes:
#             exp_ak = math.exp(np.dot(self.amodel[c]["W"], x_test) + self.amodel[c]["W0"])
#             exp_aks[c] = exp_ak
#             sum_exp_aks += exp_ak
        
#         best_posterior = 0
#         best_class = ""
#         for key, value in exp_aks.items():
#             posterior = value/sum_exp_aks
#             posteriors[key] = posterior
#             if posterior > best_posterior:
#                 best_posterior = posterior
#                 best_class = key
        
        return best_class
            
    
    def predict(self, X_test):
        self.X_test = np.array(X_test)
        predictions = np.apply_along_axis(self.predict_one, 1, self.X_test)
        return predictions
        
        
        
        
        

**Q1.2: Load data from "phone_train.picke" and train your model. List your learned model parameters in a pretty, human readable way so that the TA can easily check the correctness of your results.**

In [3]:
pgc1 = mygpc()
pgc1.fit(np.array(traindata[['x', 'y', 'z']]), np.array(traindata['label']))
pgc1.show_model_parameters()

Phoneonback
+------------------------+--------------------------+--------------------------------+----------------------------+
| mu                     | cov                      | prec                           | Others                     |
+------------------------+--------------------------+--------------------------------+----------------------------+
| [ 0.207 -0.026  9.796] | [[ 0.003 -0.002  0.003]  | [[ 943.637  469.548 -316.785]  | detcov: -18.9875162403822  |
|                        |  [-0.002  0.002 -0.002]  |  [ 469.548 1146.631  295.426]  | n: 28566                   |
|                        |  [ 0.003 -0.002  0.005]] |  [-316.785  295.426  535.768]] | prior: 0.17095459523510295 |
+------------------------+--------------------------+--------------------------------+----------------------------+

Phoneonbottom
+---------------------+--------------------------+--------------------------------+-----------------------------+
| mu                  | cov                    

**Q1.3: Load the test data from "phone_test1.pickle" and apply your predict method. List the first 20 predictions and well as the correct labels. Compute the accuracy for your model. Accuracy is the number of correct predictions divided by total number of data points.**

In [4]:
with open('phone_test1.pickle', 'rb') as fh1:
    testdata1 = pickle.load(fh1)

predictions = pgc1.predict(np.array(testdata1[['x', 'y', 'z']]))
print("First 20 predictions:")
print(predictions[:20])

test_Y = np.array(testdata1['label'])
correct_num = 0
for i, predict in enumerate(predictions):
    if predict in test_Y[i]:
        correct_num += 1
        
print("Accuracy: {}".format(correct_num/len(predictions)))
    

First 20 predictions:
['Phoneonfront' 'Phoneonbotto' 'Phoneonbotto' 'Phoneonbotto'
 'Phoneonfront' 'Phoneonright' 'Phoneonfront' 'Phoneontop' 'Phoneonfront'
 'Phoneonfront' 'Phoneonfront' 'Phoneonback' 'Phoneonleft' 'Phoneonback'
 'Phoneonbotto' 'Phoneonback' 'Phoneonright' 'Phoneonright' 'Phoneontop'
 'Phoneonleft']
Accuracy: 1.0


## Logistic Regression with L2 Regularization
(100%) We discussed the Logistic regression and its variation, Bayesian logistic regression that adopts L2 regularization. The Logistic regression with L2 regularization minimize the following error function (cf. file 0b2 linear model for classification.pdf):

λ2wTw−∑ni=1[tnlnyn+(1−tnln(1−yn)], 
where  yn=11+exp(−wTxn)  and  tn∈{0,1}  is the label value,  xn  is the feature vector, and  w  is the regression coefficient vector.

We are going to consider an extension of this model to allow different level of regularization for different regression coefficients. Consider the constant term versus other features. The coefficient of the constant term is usually not regularized in logistic regression. It is because the constant term is associated with the prior class distribution (see the discussion in generative models), and regularizing this term will force the probability of the positive class given a zero feature vector to be 0.5. This may hurt the prediction ability since the true prior class probability may indicate other more reasonable values.

Another consideration is regarding the continuous-valued features and binary-valued features. We typically normalize continuous-valued features to have zero means and unit variances but keep binary-value features untouched. It makes sense to have a single regularization value for the continuous-valued features since all of them have been normalized. Similarly, if we do not have additional information, then all binary-valued features can have the same level of regularization. However, using the same regularization coefficient for the continuous-valued and binary-valued features may not be reasonable. That is, it is often beneficial to have a regularization coefficient for the continuous-valued features, and another regularization coefficient for the binary-valued features.

The above discussion suggests that a more sophisticated way to regularize a logistic regression is to have three regularization coefficients: 0 for the constant,  a1  for continuous-valued features, and  a2  for the binary-valued features. It is possible to further refine the regularization coefficients. However, hyper-parameter tuning associated with more regularization coefficients may be costly.

To achieve this goal, we are going to consider a variation of L2-regularized logistic regression that allow different level of regularization for each coefficient. In the following discussion, we are going to use  X  to denote the feature matrix in the training data. The i-th row in  X ,  xi , is the feature vector for the i-th training data. The last column of  X  is one unless the use does not want to include the constant term.

In this model, each regression coefficient may be associated with a different regularization coefficient. Bearing with the risk of ambigulity, we (again) use the scalar  λi  to denote the regularization coefficient for  wi . The vector  w=[w1,w2,...,wD]T  is the regression coefficient vector. Let  Λ  denote the diagonal matrix that have  λi  at  Λii . Our new error function becomes:

**Q2.1 (20%) Download the Adult dataset. Clean up the dataset and create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label. All of these variables should be numpy arrays. Provide summary statistics for your training and test datasets so that TA can verify the correctness of your procedure.**

In [5]:
import pandas as pd
import requests
from io import StringIO

col_name = ["age", "workclass","fnlwgt","education","education-num","marital-status","occupation","relationship",
           "race","sex","capital-gain","capital-loss","hours-per-week","native-country","label"]

res = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")
adult_data = pd.DataFrame(np.genfromtxt(StringIO(res.text), delimiter=', ', dtype='unicode'))
adult_data.columns = col_name
adult_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [72]:
continuos_col_name = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
binary_col_name = list(set(col_name)-set(continuos_col_name))
binary_col_name.remove("label")

adult_data = adult_data.replace({'?':np.NaN}).dropna()

for col in continuos_col_name:
    adult_data[col] = pd.to_numeric(adult_data[col])

binary_df = pd.DataFrame()
for col in binary_col_name:
    df = pd.get_dummies(adult_data[col], prefix=col)
    binary_df = pd.concat([binary_df, df], axis=1)

train_dataset = pd.concat([adult_data[continuos_col_name], binary_df, adult_data[['label']]], axis=1)

deleted_feature = dict()
for col in binary_df.columns:
    if binary_df[col].sum() <= 10:
        category = col.split("_")[0]
        feature = col.split("_")[1]
        if category not in deleted_feature:
            deleted_feature[category] = []
        deleted_feature[category].append(feature)

for cat, feature_list in deleted_feature.items():
    for feature in feature_list:
        train_dataset = train_dataset[train_dataset["{}_{}".format(cat, feature)] != 1]
        train_dataset.drop("{}_{}".format(cat, feature), axis=1, inplace=True)

X_train = train_dataset.drop("label", axis=1)
Y_train = train_dataset["label"].replace({">50K":1, "<=50K":0})

print("\nSummary statistics for X_train:")
print(X_train.describe())

0

Summary statistics for X_train:
                age        fnlwgt  education-num  capital-gain  capital-loss  \
count  30152.000000  3.015200e+04   30152.000000  30152.000000  30152.000000   
mean      38.440568  1.897916e+05      10.121319   1092.370025     88.266085   
std       13.135362  1.056567e+05       2.550204   7407.547899    404.046306   
min       17.000000  1.376900e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.176248e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.784250e+05      10.000000      0.000000      0.000000   
75%       47.000000  2.376255e+05      13.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours-per-week  native-country_Cambodia  native-country_Canada  \
count    30152.000000             30152.000000           30152.000000   
mean        40.931348                 0.000597               0.003549   
std         11.979776    

In [7]:
res = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test")
adult_test = pd.DataFrame(np.genfromtxt(StringIO(res.text.split("\n",1)[1]), delimiter=', ', dtype='unicode'))
adult_test.columns = col_name

In [8]:
adult_test = adult_test.replace({'?':np.NaN}).dropna()

for col in continuos_col_name:
    adult_test[col] = pd.to_numeric(adult_test[col])
    
test_binary_df = pd.DataFrame()
for col in binary_col_name:
    df = pd.get_dummies(adult_test[col], prefix=col)
    test_binary_df = pd.concat([test_binary_df, df], axis=1)

# train_test_diff = set(binary_df.columns).symmetric_difference(set(test_binary_df.columns))
# if len(train_test_diff) > 0:
#     print(train_test_diff)

# found that there is no Holand-Netherlands in testing data
    
test_dataset = pd.concat([adult_test[continuos_col_name], test_binary_df, adult_test[['label']]], axis=1)

for cat, feature_list in deleted_feature.items():
    for feature in feature_list:
        if feature == "Holand-Netherlands":
            continue
        test_dataset = test_dataset[test_dataset["{}_{}".format(cat, feature)] != 1]
        test_dataset.drop("{}_{}".format(cat, feature), axis=1, inplace=True)

X_test = test_dataset.drop("label", axis=1).reindex(columns=X_train.columns)
Y_test = test_dataset["label"].replace({">50K.":1, "<=50K.":0})

print("\nSummary statistics for X_test:")
print(X_test.describe())


Summary statistics for X_test:
                age        fnlwgt  education-num  capital-gain  capital-loss  \
count  15055.000000  1.505500e+04   15055.000000  15055.000000  15055.000000   
mean      38.769711  1.896046e+05      10.112255   1120.188907     89.071471   
std       13.381046  1.056274e+05       2.558629   7704.274825    406.347469   
min       17.000000  1.349200e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.166450e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.779510e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.385810e+05      13.000000      0.000000      0.000000   
max       90.000000  1.490400e+06      16.000000  99999.000000   3770.000000   

       hours-per-week  native-country_Cambodia  native-country_Canada  \
count    15055.000000             15055.000000           15055.000000   
mean        40.950714                 0.000531               0.003720   
std         12.064465       

**Q2.2 (20%) Derive the gradient and hession matrix for the new E(w).**

$\nabla E(w) = \Lambda w + \Phi ^ T (y-t)$, where y = $\sigma(w^T \phi_n)$   
$\nabla \nabla E(w) = \Lambda + \Phi^T R \Phi$, where R is a diagonal N x N matrix, with $R_(nn) = y_n(1-y_n)$

**Q2.3 (30%) Create your own mylogistic_l2 class. Show the learned w as well as test accuracy for the cases below. If w is too long for you, show selected w for continuous-valued, binary-valued, and the constant term.**   
* Case 1: lambda = 1 for all coefficients   
* Case 2: lambda = 1 for all but the intercept, no regularization for incercept term.   
* Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [110]:
import math
import numpy as np

class mylogistic_l2():
    
    def __init__(self, reg_vec:list, max_iter:int, tol, add_intercept:bool):
        
        self.reg_vec = np.array(reg_vec)
        self.reg_matrix = np.diag(self.reg_vec)
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        self.X_train = None
        self.Y_train = None
        
        self.w_best = None
        
    def fit(self, x, y):
        self.X_train = x # a N x M matrix
        self.Y_train = y.reshape((len(y), 1)) # t is a N x 1 matrix
        
        if self.add_intercept is True:
            const = np.zeros((len(self.X_train), 1)) + 1
            self.X_train = np.append(self.X_train, const, axis=1)
        
        self.train()
    
    def sigmoid(self, x):
            return(1 / (1+math.exp(-x)) )
            
    def train(self):
        
            
        def get_gradient_hessian(w):
            """compute the gradient and hessian of the error function with corresponding vector w"""
            
            w = w.reshape((len(w), 1)) # w -> M x 1 matrix
            pred = np.matmul(self.X_train, w) # (N x M)(M x 1)
            pred_sigmoid = np.apply_along_axis(func1d=self.sigmoid, arr=pred, axis=1).reshape((len(pred),1))
            
            # compute gradient
            reg_gradient = np.matmul(self.reg_matrix, w) # (M x M)(M x 1)
            err_gradient = np.matmul(np.transpose(self.X_train), pred_sigmoid - self.Y_train) # (M x N)(N x 1)
            gradient = reg_gradient + err_gradient #  a (M x 1) matrix
            
            # compute hessian
#             R = np.diag((pred_sigmoid*(1-pred_sigmoid)).flatten())
            R = (pred_sigmoid*(1-pred_sigmoid)).flatten()
            err_hessian = np.matmul(np.multiply(np.transpose(self.X_train), R), self.X_train)
            hessian = self.reg_matrix + err_hessian
            
            return((gradient, hessian))
        
        def get_W0():
            """use the closed-form solution of ridge regression"""
            
            b = self.reg_vec.mean()
            bI = np.diag(b * np.array([1 for i in range(len(self.reg_vec))]))
            
            #(X^T x X + bI)
            pre = np.matmul(np.transpose(self.X_train), self.X_train) + bI
            
            try:
                pre_inv = np.linalg.inv(pre)
            except numpy.linalg.LinAlgError:
                raise ValueError("Invertible Error while generating W0.")
            
            W0 = np.matmul(np.matmul(pre_inv, np.transpose(self.X_train)), self.Y_train)
            return(W0)
        
        def error_function(w):
            """return the error value with corresponding w"""
            
            def one_row_error(x):
                t = int(x[-1])
                x = x[:-1]

                x = x.reshape((len(x), 1))
                y = self.sigmoid(np.matmul(np.transpose(w), x))
                
                return (t*math.log(y) + (1-t)*math.log(1-y))
                
            w = w.reshape((len(w), 1)) # w -> M x 1 matrix
            error_list = np.apply_along_axis(arr=np.append(self.X_train, self.Y_train, axis=1), 
                                             func1d=one_row_error, axis=1)
            
            return(error_list.sum() * -1)
        
        w_old = get_W0()
        error_old = error_function(w_old)
        
        for i in range(self.max_iter):
            gradient, hessian = get_gradient_hessian(w_old)
            try:
                hessian_inv = np.linalg.inv(hessian)
            except numpy.linalg.LinAlgError:
                raise ValueError("Invertible Error while inverting hessian on {}th iteration".format(i+1))
            
            w_new = w_old - np.matmul(hessian_inv, gradient)
            error_new = error_function(w_new)
            
            if (error_new >= error_old):
#                 print("Best w found after {}th iteration".format(i))
                break
            
            w_old = w_new
            
            if (error_old - error_new) < self.tol:
#                 print("Best w found after {}th iteration".format(i+1))
                break
            
            error_old = error_new
            
        self.w_best = w_old
#         print("Finish training.")
    
    def predict_single(self, x_test):
        x_test = x_test.reshape((len(x_test), 1))
        c1_probability = self.sigmoid(np.matmul(np.transpose(self.w_best), x_test))
        
        prediction = 1 if c1_probability > 0.5 else 0
        return(prediction)
    
    def predict(self, X_test):
        self.X_test = X_test
        if self.add_intercept is True:
            const = np.zeros((len(X_test), 1)) + 1
            self.X_test = np.append(self.X_test, const, axis=1)
        
        predictions = np.apply_along_axis(self.predict_single, 1, self.X_test)
        return(predictions)

<br>   
* Case 1: lambda = 1 for all coefficients  

In [51]:
M = X_train.shape[1]
lambda_vec = [1 for i in range(M+1)]
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
logic1.fit(np.array(X_train), np.array(Y_train))

Best w found after 8th iteration
Finish training.


In [59]:
predictions_1 = logic1.predict(np.array(X_test))
correct_num = 0

observations = np.array(Y_test)
for i in range(len(Y_test)):
    if predictions_1[i] == observations[i]:
        correct_num += 1

print("Accuracy for case 1: {}".format(correct_num/len(Y_test)))

Accuracy for case 1: 0.8481567585519761


<br>   
* Case 2: lambda = 1 for all but the intercept, no regularization for incercept term.

In [57]:
M = X_train.shape[1]
lambda_vec = [1 for i in range(M+1)]
lambda_vec[-1] = 0
logic2 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
logic2.fit(np.array(X_train), np.array(Y_train))

Best w found after 8th iteration
Finish training.


In [58]:
predictions_2 = logic2.predict(np.array(X_test))
correct_num = 0

observations = np.array(Y_test)
for i in range(len(Y_test)):
    if predictions_2[i] == observations[i]:
        correct_num += 1

print("Accuracy for case 2: {}".format(correct_num/len(Y_test)))

Accuracy for case 2: 0.8478910660909996


<br>   
* Case 3:  lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [81]:
M = X_train.shape[1]
lambda_vec = [1 for i in range(M+1)]
index = len(continuos_col_name)
for i in range(index, len(lambda_vec)):
    lambda_vec[i] = 0.5

lambda_vec[-1] = 0

logic3 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
logic3.fit(np.array(X_train), np.array(Y_train)) 

Best w found after 8th iteration
Finish training.


In [83]:
predictions_3 = logic3.predict(np.array(X_test))
correct_num = 0

observations = np.array(Y_test)
for i in range(len(Y_test)):
    if predictions_3[i] == observations[i]:
        correct_num += 1

print("Accuracy for case 3: {}\n".format(correct_num/len(Y_test)))

Accuracy for case 3: 0.8477582198605115



Q2.4 (10%) Further split the training data into subtraining (90%) and tuning (10%) to search for the best hyper-parameters. Set the regularization coefficient for the constant term to zero. Allow different regularizations for continuous-valued and binary-valued features. Let  a1  and  a2  denote the regularization coefficients for continuous-valued and binary-valued features. Search the best  a1  and  a2  and report the test accuracy using the best hyper-parameters. You should follow the following procedure to search for the best hyper-parameters.

Choose a set of grids among a reasonable range. For example, 10 grids in [0.01, 100].
Conduct grid search with the constraint that  a1=a2 . Record the best value  a∗1  and  a∗2 .
Fix  a1=a∗1 , and search  a2  for the best value, call the result the new  a∗2 .
Fix  a2=a∗2 , and search  a1  for the best value.
Report the selected  a1  and  a2 .
Train a model using the selected hyper-parameters, and report the test accuracy.

In [111]:
from sklearn.model_selection import train_test_split

def get_lambda_vec(a1, a2, a3=0):
    con = [a1 for i in range(len(continuos_col_name))]
    bin = [a2 for i in range(X_train.shape[1] - len(continuos_col_name))]
    return(con+bin+[a3])

def accuracy_score(pred, observ):
    correct_num = 0
    for i in range(len(observ)):
        if pred[i] == observ[i]:
            correct_num += 1
    return(correct_num/len(observ))

X_subtrain, X_tuning, Y_subtrain, Y_tuning = train_test_split(np.array(X_train), np.array(Y_train), test_size=0.1)
a_list = np.logspace(-2, 2, 10) 
#[  0.01 ,   0.028,   0.077,   0.215,   0.599,   1.668,   4.642, 12.915,  35.938, 100. ]

best_accuracy = 0
best_a = None
for a in a_list:
    lambda_vec = get_lambda_vec(a,a)
    logic = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
    logic.fit(np.array(X_subtrain), np.array(Y_subtrain))
    predictions = logic.predict(np.array(X_tuning))
    accuracy = accuracy_score(predictions, np.array(Y_tuning))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_a = a

print("best a1=a2= {}".format(best_a))        
        
a1 = best_a
best_accuracy = 0
best_a2 = None
for a in a_list:
    lambda_vec = get_lambda_vec(a1,a)
    logic = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
    logic.fit(np.array(X_subtrain), np.array(Y_subtrain))
    predictions = logic.predict(np.array(X_tuning))
    accuracy = accuracy_score(predictions, np.array(Y_tuning))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_a2 = a

print("best a2= {}".format(best_a2))         
        
best_accuracy = 0
best_a1 = None
for a in a_list:
    lambda_vec = get_lambda_vec(a,best_a2)
    logic = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
    logic.fit(np.array(X_subtrain), np.array(Y_subtrain))
    predictions = logic.predict(np.array(X_tuning))
    accuracy = accuracy_score(predictions, np.array(Y_tuning))
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_a1 = a

print("best a1= {}".format(best_a1))

best a1=a2= 1.6681005372000592
best a2= 1.6681005372000592
best a1= 0.01


In [118]:
lambda_vec = get_lambda_vec(best_a1,best_a2)
logic4 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
logic4.fit(np.array(X_train), np.array(Y_train))
predictions = logic4.predict(np.array(X_test))
accuracy = accuracy_score(predictions, np.array(Y_test))

print("Accuracy for case 4: {}\n".format(accuracy))

Accuracy for case 4: 0.848090335436732



Q2.5 (20%) Use sklearn.linear_model.LogisticRegression to train and test the model (including hyper-parameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [124]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from warnings import simplefilter # import warnings filter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
# grid={"C":np.logspace(-2,2,10), "solver":["newton-cg"]}
# grid={"C":np.logspace(-2,2,10)}
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(np.array(X_train),np.array(Y_train))

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 1.0, 'penalty': 'l1'}
accuracy : 0.8474728044574158


In [133]:
logreg_official=LogisticRegression(penalty='l1', C=1.0)
logreg_official.fit(np.array(X_train),np.array(Y_train))
accuracy = logreg_official.score(np.array(X_test), np.array(Y_test))
print("Accuracy for case 5: {}".format(accuracy))

Accuracy for case 5: 0.8482231816672202


By using the sklearn model, I get the highest Accuracy 0.84822.   
However, the model is using l1 penalty, not l2 penalty like mylogistic_l2